In [ ]:
import os
import gradio as gr
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(
    temperature=0.3,
    model_name="llama3-8b-8192"
)


C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ERROR:crewai.telemetry.telemetry:HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000019E66CD4690>, 'Connection to telemetry.crewai.com timed out. (connect timeout=30)'))


In [2]:

sentiment_agent = Agent(
    role="Sentiment Analyst",
    goal="Analyze the overall sentiment of the social media post",
    backstory="An expert in detecting emotional tones like positive, negative, or neutral from user-generated content.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

emotion_agent = Agent(
    role="Emotion Classifier",
    goal="Detect emotions like joy, anger, sadness, etc., in the post",
    backstory="Specializes in mapping language to emotional signals for better psychological profiling.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

hashtag_agent = Agent(
    role="Hashtag Strategist",
    goal="Analyze and improve the use of hashtags",
    backstory="Focused on extracting and optimizing hashtags for reach and engagement.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

engagement_agent = Agent(
    role="Engagement Predictor",
    goal="Predict the likely engagement (likes, shares, comments) based on the post",
    backstory="Has worked with social media firms to estimate user engagement through text features.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

rewriter_agent = Agent(
    role="Engagement Optimizer",
    goal="Rewrite the post to improve emotional appeal and engagement potential",
    backstory="A content copywriter who specializes in emotionally intelligent and viral content design.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)


Define Tasks

In [3]:

def build_tasks(user_post):
    task1 = Task(
        description=f"Analyze this social media post for sentiment (Positive, Neutral, Negative):\n\n{user_post}",
        expected_output="Label and brief reasoning for the sentiment.",
        agent=sentiment_agent
    )

    task2 = Task(
        description=f"Classify the emotions in the following post. Return top 2 emotions and justify:\n\n{user_post}",
        expected_output="List of emotions like joy, sadness, anger, surprise, disgust.",
        agent=emotion_agent
    )

    task3 = Task(
        description=f"Extract and analyze hashtags from the post. Suggest 2-3 more relevant hashtags:\n\n{user_post}",
        expected_output="List of hashtags and replacements.",
        agent=hashtag_agent
    )

    task4 = Task(
        description=f"Predict engagement levels (high, medium, low) for the post below:\n\n{user_post}",
        expected_output="Engagement label with short explanation.",
        agent=engagement_agent
    )

    task5 = Task(
        description=f"Rewrite the following post to improve engagement and emotional appeal:\n\n{user_post}",
        expected_output="A revised version of the post with higher emotional impact.",
        agent=rewriter_agent
    )

    return [task1, task2, task3, task4, task5]


Gradio interface

In [ ]:


def run_crew_analysis(user_post):
    tasks = build_tasks(user_post)
    crew = Crew(
        agents=[sentiment_agent, emotion_agent, hashtag_agent, engagement_agent, rewriter_agent],
        tasks=tasks,
        process=Process.sequential,
        verbose=True
    )
    result = crew.kickoff()
    return result

iface = gr.Interface(
    fn=run_crew_analysis,
    inputs=gr.Textbox(lines=6, label="Enter your social media post for analysis"),
    outputs=gr.Textbox(label="🧠 Multi-Agent Analysis Report"),
    title="📊 Social Media Intelligence AI Crew",
    description="Enter a single social media post. This tool will analyze sentiment, emotion, hashtags, engagement, and rewrite the post for maximum impact using LLaMA3 (Groq)."
)

if __name__ == "__main__":
    iface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9f49ed24-2ee6-4fd6-ad4d-94573d941096                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Sentiment Analyst                                                                                       │
│                                                                                                                 │
│  Task: Analyze this social media post for sentiment (Positive, Neutral, Negative):                              │
│                                                                                                                 │
│  Just crushed a morning 10K run in the rain! 🌧️🏃‍♂️ Feeling unstoppable and so alive. #grindmode #runnerslife     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call.    │
│  You passed model=llama3-8b-8192                                                                                │
│   Pass model as E.g. For 'Huggingface' inference endpoints pass in                                              │
│  `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 742a56ac-cb02-4505-aae9-d31f7fe4fc0d                                                                     │
│  Agent: Sentiment Analyst                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 9f49ed24-2ee6-4fd6-ad4d-94573d941096                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Traceback (most recent call last):
  File "C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\gradio\blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
 